In [ ]:
# imports
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
import tensorflow as tf
import tensorflow.keras as K
import numpy as np
import pandas as pd
import os
import glob
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
# get directories
main_direc = os.getcwd()
images_dir = os.path.join(main_direc, 'data/images/images')

# csv location
artist_csv_loc = os.path.join(main_direc, 'data/artists.csv')

## Loading Data
The below section loads in the data, making necessary preprocess changes.
Generators are used to augment the data.

### Hyperparameter Section for data preprocessing
The following code block allows you to set different hyperparams for loading in
the images.

In [ ]:
"""
Set hyperparams for the number of classes and image generators
"""

IMG_WIDTH = 299
IMG_HEIGHT = 299
BATCH_SIZE = 64
NUM_ARTISTS = 10 # this is 11 to get to 10 classes, can be changed...

### DataFrame management
The following code block loads the artists csv into pandas dataframe, sorts by
number of paintings, and makes a dataframe with the top 10 artists by painting
count, to give us the most amount of data possible.

In [ ]:
# Collecting Needed Images
artists = pd.read_csv(artist_csv_loc)

# Creating a dataframe with the top 10 artists by number of paintings
artists_sort = artists.sort_values(by=['paintings'], ascending=False)

# add one to reach the first 10 classes
artists_top = artists_sort.head(NUM_ARTISTS + 1)
print(artists_top)

# Images
artists_dir = os.listdir(images_dir) # Files are named after each artists

# Images DataFrame
artists_top_name = artists_top['name'].str.replace(' ', '_').values

images_df = pd.DataFrame()
for name in artists_top_name:
    images_df = pd.concat([images_df, pd.DataFrame(data={'Path': glob.glob('data/images/images/' + name + '/*'), 'Name': name})], ignore_index=True)

print(images_df)

train_df = images_df.sample(frac=0.8, random_state=200)
test_df = images_df.drop(train_df.index)

if K.backend.image_data_format() == 'channels_first':
    input_shape = (3, IMG_HEIGHT, IMG_WIDTH)
else:
    input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)

### Building generators
The next code block builds generators for augmenting the data.

In [ ]:
"""
Build generators
"""

train_generator = ImageDataGenerator(rescale=1.0 / 255,
                                    rotation_range=20,
                                    zoom_range=0.05,
                                    width_shift_range=0.05,
                                    height_shift_range=0.05,
                                    shear_range=0.05,
                                    horizontal_flip=True,
                                    fill_mode="nearest",
                                    validation_split=0.15,
                                    preprocessing_function=preprocess_input
                                    )

test_generator = ImageDataGenerator(rescale=1.0 / 255, preprocessing_function=preprocess_input)

train_gen = train_generator.flow_from_dataframe(
    train_df,
    shuffle=True,
    x_col='Path',
    y_col='Name',
    class_mode='categorical',
    subset="training",
    batch_size=BATCH_SIZE,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    seed=42
)

valid_gen = train_generator.flow_from_dataframe(
    train_df,
    subset="validation",
    shuffle=True,
    x_col='Path',
    y_col='Name',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    seed=42
)

test_gen = test_generator.flow_from_dataframe(
    test_df,
    x_col='Path',
    batch_size=1,
    shuffle=False,
    class_mode=None,
    target_size=(IMG_HEIGHT, IMG_WIDTH)
)

# Set the amount of steps for training, validation, and testing data
# based on the batch size
steps_train = train_gen.n//train_gen.batch_size
steps_valid = valid_gen.n//valid_gen.batch_size
steps_test = test_gen.n//test_gen.batch_size

## Clay's Model
The following model was designed by Clay Kaufmann. It uses Inception V3 as a
base line, and makes modifications from there.

### Hyperparameter Block
Set different hyperparameters for the model with the following block.

In [ ]:
"""
Hyperparameters here:
"""

N_EPOCHS = 10
LEARNING_RATE = 0.001 # 0.001 is the default for Adam set by TensorFlow
OPTIMIZER = tf.optimizers.Adam(learning_rate=LEARNING_RATE)
LOSS_FUNCTION = tf.losses.CategoricalCrossentropy(from_logits=False)

In [ ]:
"""
Build the model
"""

# set the input for VGG
inp = Input(shape=(IMG_HEIGHT,IMG_WIDTH,3))

# load model
base_model = InceptionV3(include_top=False, input_tensor=inp, pooling='max', weights='imagenet')

# set base model to not be trainable
base_model.trainable = False

# create sequential model
final_model = tf.keras.Sequential()

# add the base model in
final_model.add(base_model)

final_model.add(K.layers.Flatten())
final_model.add(K.layers.BatchNormalization())
final_model.add(Dense(256, activation='relu'))
final_model.add(K.layers.Dropout(0.6))

# this needed to be the number of artists - 1
final_model.add(Dense(NUM_ARTISTS, activation='softmax'))

final_model.summary()

# compile model
final_model.compile(
  optimizer=OPTIMIZER,
  loss=LOSS_FUNCTION,
  metrics=['accuracy']
)

In [ ]:
"""
Train the model
"""

# create a checkpoint for the model
checkpt = ModelCheckpoint(filepath='clay_trained_model.hdf5', save_best_only=True, verbose=1)

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, mode='min')

# Fit the model
final_model.fit_generator(
    generator = train_gen,
    steps_per_epoch=steps_train,
    validation_data = valid_gen,
    validation_steps = steps_valid,
    verbose=1,
    epochs=N_EPOCHS,
    callbacks=[checkpt, early_stop]
)

### Metrics Section
In the following codeblocks, metrics are used and plots are created to show how
the model performed.  
A confusion matrix is creating, using Seaborn's heatmap feature. A
classification report from sklearn metrics is also run.

In [ ]:
"""
Make predictions
"""

# Load model weights
final_model.load_weights('clay_trained_model.hdf5')

# make predictions using the test set
predict = final_model.predict_generator(test_gen, steps=steps_test, verbose=1)

# collect the actual index of the predicted class
predicted_class = np.argmax(predict, axis=1)

In [ ]:
"""
Produce result metrics
"""

# collect the actual classes of the predictions
r = dict(train_gen.class_indices.items())
y_true = test_df['Name']
y_true = [r[k] for k in y_true]

# create confusion matrix
cm = confusion_matrix(y_true, predicted_class)
sns.heatmap(cm, annot=True)

# print classification report
print(classification_report(y_true, predicted_class))